In [1]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

def extract_text_from_pdf(pdf_path, output_txt="extracted_text.txt"):
    """
    Extracts text from a PDF and applies OCR where necessary.
    :param pdf_path: Path to the PDF file.
    :param output_txt: Path to save extracted text.
    :return: Extracted text.
    """
    doc = fitz.open(pdf_path)
    full_text = ""

    for page_num, page in enumerate(doc):
        text = page.get_text("text")
        if not text.strip():  # If no embedded text, use OCR
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            text = pytesseract.image_to_string(img)

        full_text += f"\n\n--- Page {page_num+1} ---\n\n" + text

    # Save the text to a file
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write(full_text)

    return full_text

# Example usage
pdf_path = r"E:\Btech_AI\Intern\ocrpro\Phable CAM Final.pdf"  # Replace with actual PDF
extracted_text = extract_text_from_pdf(pdf_path)
print("✅ Text extraction complete. Saved in extracted_text.txt.")


✅ Text extraction complete. Saved in extracted_text.txt.


In [3]:
import pdfplumber
import pandas as pd

def extract_tables_with_pdfplumber(pdf_path):
    """
    Extracts tables using pdfplumber.
    :param pdf_path: Path to the PDF file.
    :return: List of DataFrames.
    """
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                df = pd.DataFrame(table).replace("", None)
                if df.shape[1] > 1 and df.shape[0] > 1:  # Ignore non-tables
                    tables.append(df)
    
    return tables

# Extract tables
tables_pdfplumber = extract_tables_with_pdfplumber(pdf_path)
print(f"✅ Extracted {len(tables_pdfplumber)} tables using pdfplumber.")


✅ Extracted 46 tables using pdfplumber.


In [23]:
def align_columns(df):
    """
    Attempts to realign misaligned columns in a table.
    :param df: DataFrame with potential misalignment.
    :return: Re-aligned DataFrame.
    """
    if df.empty or df.shape[1] == 0:  # ✅ Handle empty tables
        return None

    df = df.dropna(how='all')  # Remove empty rows

    if df.empty or df.shape[1] == 0:  # ✅ Check again after dropping empty rows
        return None

    # If there’s a row with more columns, use it as the header
    try:
        max_cols = max(df.apply(lambda x: x.count(), axis=1))  # Get the row with most non-null values
        for idx, row in df.iterrows():
            if row.count() == max_cols:
                df.columns = row
                df = df.iloc[idx + 1:]  # Remove the header row from data
                break
    except ValueError:
        return None  # ✅ Return None if column alignment fails

    return df.reset_index(drop=True).fillna("")

# ✅ Filter out empty tables before processing
aligned_tables = [align_columns(df) for df in tables_pdfplumber if df is not None]
aligned_tables = [df for df in aligned_tables if df is not None]  # Remove None values

def ensure_unique_columns(df):
    """
    Ensures all column names in the DataFrame are unique.
    :param df: Pandas DataFrame.
    :return: DataFrame with unique column names.
    """
    if df.empty:
        return df
    
    seen = {}
    new_columns = []
    
    for col in df.columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_columns.append(col)

    df.columns = new_columns
    return df

# Apply unique column renaming to all extracted tables
aligned_tables = [ensure_unique_columns(df) for df in aligned_tables]

print(f"✅ {len(aligned_tables)} tables successfully aligned.")


def merge_multiline_headers(df, max_header_rows=2):
    """
    Merges multi-line headers into a single row.
    :param df: Pandas DataFrame.
    :param max_header_rows: Number of rows to consider as headers.
    :return: DataFrame with merged headers.
    """
    if df.empty or df.shape[1] == 0:
        return df

    # Limit to max_header_rows and fill empty header spaces
    header_rows = df.iloc[:max_header_rows].fillna("")
    
    # Merge headers into a single row
    merged_header = [" ".join(str(col).strip() for col in row if str(col).strip()) for row in zip(*header_rows.values)]
    
    # Assign new headers and remove original header rows
    df.columns = merged_header
    df = df.iloc[max_header_rows:].reset_index(drop=True)
    
    return df

# Apply to all extracted tables
aligned_tables = [merge_multiline_headers(df) for df in aligned_tables]



✅ 41 tables successfully aligned.


In [25]:
import ollama
import json
import pandas as pd

def refine_tables_with_llama(tables):
    """
    Uses a local LLaMA model via Ollama to refine table structure.
    :param tables: List of Pandas DataFrames.
    :return: List of refined DataFrames.
    """
    tables_json_safe = [df.to_dict(orient="records") for df in tables]

    prompt = (
        "You are an AI that structures tables extracted from PDFs. "
        "Format them into structured JSON without extra text. "
        "Ensure the output is in this format: "
        '{"tables": [{"columns": ["col1", "col2"], "rows": [["row1_col1", "row1_col2"], ...]}]}. '
        "Here are the extracted tables:\n"
        f"{json.dumps(tables_json_safe)}"
    )

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])

    try:
        # ✅ Extract response safely
        response_text = response["message"]["content"].strip()

        if not response_text:  # ✅ Check for empty response
            raise ValueError("LLaMA returned an empty response.")

        # ✅ Remove Markdown formatting if present
        if "```json" in response_text:
            response_text = response_text.split("```json")[1].split("```")[0].strip()

        refined_tables_json = json.loads(response_text)  

        refined_tables = [pd.DataFrame(table["rows"], columns=table["columns"]) for table in refined_tables_json["tables"]]

    except Exception as e:
        print(f"⚠ Error parsing LLaMA output: {e}")
        refined_tables = tables  # Use original tables if parsing fails

    return refined_tables


In [ ]:
import ollama

def refine_tables_with_mistral(tables):
    """
    Uses Mistral via Ollama to refine table structure.
    :param tables: List of Pandas DataFrames.
    :return: List of refined DataFrames.
    """
    tables_json_safe = [df.to_dict(orient="records") for df in tables]

    prompt = (
        "You are an AI that structures tables extracted from PDFs. "
        "Format them into structured JSON without extra text. "
        "Ensure the output is in this format: "
        '{"tables": [{"columns": ["col1", "col2"], "rows": [["row1_col1", "row1_col2"], ...]}]}. '
        "Here are the extracted tables:\n"
        f"{json.dumps(tables_json_safe)}"
    )

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])

    try:
        response_text = response["message"]["content"].strip()

        if not response_text:
            raise ValueError("Mistral returned an empty response.")

        if "```json" in response_text:
            response_text = response_text.split("```json")[1].split("```")[0].strip()

        refined_tables_json = json.loads(response_text)  

        refined_tables = [pd.DataFrame(table["rows"], columns=table["columns"]) for table in refined_tables_json["tables"]]

    except Exception as e:
        print(f"⚠ Error parsing Mistral output: {e}")
        refined_tables = tables  

    return refined_tables

# Test both models
refined_tables_llama = refine_tables_with_llama(aligned_tables)
refined_tables_mistral = refine_tables_with_mistral(aligned_tables)

# Compare results
print("✅ Table structuring complete for LLaMA & Mistral.")


C:\Users\chira\AppData\Local\Temp\ipykernel_8664\1766809467.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  tables_json_safe = [df.to_dict(orient="records") for df in tables]
C:\Users\chira\AppData\Local\Temp\ipykernel_8664\1766809467.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  tables_json_safe = [df.to_dict(orient="records") for df in tables]
C:\Users\chira\AppData\Local\Temp\ipykernel_8664\1766809467.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  tables_json_safe = [df.to_dict(orient="records") for df in tables]
C:\Users\chira\AppData\Local\Temp\ipykernel_8664\1766809467.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  tables_json_safe = [df.to_dict(orient="records") for df in tables]
C:\Users\chira\AppData\Local\Temp\ipykernel_8664\1766809467.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  ta

In [ ]:
import os

output_folder = "output_tables"
os.makedirs(output_folder, exist_ok=True)

def save_tables_to_excel(tables, output_excel):
    """
    Saves structured tables in an Excel file (one sheet per table).
    :param tables: List of DataFrames.
    :param output_excel: Path to the Excel file.
    """
    if not tables:
        print("⚠ No tables to save.")
        return

    with pd.ExcelWriter(output_excel) as writer:
        for idx, df in enumerate(tables):
            df.to_excel(writer, sheet_name=f"Table_{idx+1}", index=False)

# Save structured tables
output_excel = os.path.join(output_folder, "final_tables.xlsx")
save_tables_to_excel(refined_tables, output_excel)
print(f"✅ Final structured tables saved in {output_excel}.")
